In [1]:
import pandas as pd

In [26]:
df = pd.read_csv("input.tsv", sep="\t", na_values=["(null)"])
df = df[df["scr_id"].notna()]
df = df[df["scr_id"].map(lambda s: s.startswith("SCR_"))]
df[df["scr_id"] == df["Canonical_ID"]]

for column in [
    "type",
    "Supercategory",
    "Related_Disease",
    "Species",
    "Related_Application",
    "Funding_Information",
    "Publication_Link",
    "Old_URLs",
    "Social_URLs",
    "Supporting_Agency",
    "Editorial_Note",
    "Canonical_ID",
]:
    del df[column]

all_na = {column for column in df.columns if df[column].isna().all()}
for column in all_na:
    del df[column]

df.to_csv("1.tsv", sep="\t", index=False)
df

/var/folders/vp/n7_7v49n7dl7ln9bs8r0wbt80000gn/T/ipykernel_56812/2646654916.py:1: DtypeWarning: Columns (25,26,35,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("input.tsv", sep='\t', na_values=["(null)"])


,scr_id,original_id,parent_organization_id,Resource_Name,Defining_Citation,Additional_Resource_Types,Synonyms,Abbreviation,Keywords,Resource_URL,...,Alternate_URLs,Terms_Of_Use_URLs,Alternate_IDs,Comment,License,relationship_strings,resources_scr_ids,RRID_Identifier_Pattern,Unnamed: 39,Unnamed: 40
0,SCR_000002,nlx_152901,"SCR_001373,SCR_000824",monarch-ontologies,NaN,"ontology,controlled vocabulary,data or informa...",monarch.owl,Monarch Ontologies,phenotype,http://purl.obolibrary.org/obo/upheno/monarch.owl,...,NaN,NaN,nlx_152901,NaN,NaN,is_related_to|has_parent_organization|has_pare...,SCR_003214|SCR_001373|SCR_001373,NaN,NaN,NaN
1,SCR_000005,nif-0000-00023,NaN,Neuroshare - Open data specifications and soft...,NaN,"software resource,software toolkit,software li...",Neuroshare.org,NaN,"data management, development, interoperability...",http://neuroshare.sourceforge.net/index.shtml,...,NaN,NaN,nif-0000-00023,NaN,NaN,is_related_to,SCR_004951,NaN,NaN,NaN
2,SCR_000009,OMICS_05617,SCR_006442,ncdfFlow,NaN,software resource,ncdfFlow: A package that provides ncdf based s...,NaN,"software package, mac os x, unix/linux, window...",http://www.bioconductor.org/packages/release/b...,...,NaN,NaN,OMICS_05617,NaN,NaN,is_listed_by|has_parent_organization,SCR_002250|SCR_006442,NaN,NaN,NaN
3,SCR_000010,nif-0000-00107,NaN,Computational Neuroscience on the Web,NaN,"portal,topical portal,data or information reso...",NaN,NaN,"computational, neuroscience, compartmental ,mo...",https://compneuroweb.com/,...,NaN,NaN,nif-0000-00107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SCR_000011,SciRes_000155,SCR_008960,Leica DMRE Fluorescence Microscope,NaN,instrument resource,Fluorescence Leica DMRE microscope,Leica DMRE microscope,"microscope, fluorescence, imaging, hardware, i...",http://www.uu.nl/faculty/veterinarymedicine/EN...,...,NaN,NaN,SciRes_000155,NaN,NaN,is_listed_by|has_parent_organization,SCR_018018|SCR_008960,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20180,SCR_024533,SCR_024533,NaN,MediBeacon Transdermal GFR Monitor,PMID:30394397,instrument resource,NaN,NaN,"Transdermal, Glomerular Filtration Rate, monit...",https://www.medibeacon.com/science/preclinical/,...,NaN,NaN,NaN,Commercially available,NaN,NaN,NaN,NaN,NaN,NaN
20181,SCR_024534,SCR_024534,NaN,3D ImageJ Suite,NaN,"software application,software resource",NaN,NaN,"enhance 3D capabilities of ImageJ,",https://imagej.net/plugins/3d-imagej-suite/,...,NaN,NaN,NaN,NaN,NaN,works_with,SCR_018680,NaN,NaN,NaN
20182,SCR_024535,SCR_024535,NaN,Leica TP1020 Automatic Benchtop Tissue Process...,NaN,instrument resource,Leica TP1020 tissue processor,NaN,"tissue processor,specimen processing ,",https://www.leicabiosystems.com/de-de/histolog...,...,NaN,NaN,Model_Number_TP1020,"Manufacturer:Leica, Brand:Leica",NaN,NaN,NaN,NaN,NaN,NaN
20183,SCR_024536,SCR_024536,NaN,10X Genomics Chromium iX Single Cell Analyzer,NaN,instrument resource,"Chromium iX Single Cell Analyzer, Chromium iX,...",NaN,"lower throughput single cell analysis, instrum...",https://www.10xgenomics.com/instruments/chromi...,...,NaN,NaN,NaN,Commercially available,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
rrid_df = df[
    df["RRID_Identifier_Pattern"].notna()
    & df["RRID_Identifier_Pattern"].map(lambda s: isinstance(s, str) and s.startswith("RRID:"))
].copy()
rrid_df["prefix"] = rrid_df["RRID_Identifier_Pattern"].map(
    lambda s: s.removeprefix("RRID:").removesuffix("_")
)
rrid_df = rrid_df.sort_values("prefix")
del rrid_df["RRID_Identifier_Pattern"]
rrid_df = rrid_df.set_index("prefix")
rrid_df["bioregistry"] = rrid_df.index.map(bioregistry.normalize_prefix)
first = "Resource_Name", "bioregistry"
rrid_df = rrid_df[[*first, *(c for c in rrid_df.columns if c not in first)]]
rrid_df.to_csv("2.tsv", sep="\t")
rrid_df

,Resource_Name,bioregistry,scr_id,original_id,parent_organization_id,Defining_Citation,Additional_Resource_Types,Synonyms,Abbreviation,Keywords,...,Twitter_Handle,Alternate_URLs,Terms_Of_Use_URLs,Alternate_IDs,Comment,License,relationship_strings,resources_scr_ids,Unnamed: 39,Unnamed: 40
prefix,,,,,,,,,,,,,,,,,,,,,
AB,Antibody Registry,None,SCR_006397,nif-0000-07730,SCR_002894,NaN,"storage service resource,data or information r...","AntibodyRegistry, The Antibody Registry, ABReg...",NaN,"RIN, Resource Information Network, antibody, r...",...,NaN,NaN,https://antibodyregistry.org/terms-and-conditions,"nif-0000-07730, OMICS_01768, biodbcore-000182",Access: open; Copyrights - http://neuinfo.org/...,CC BY,is_used_by|is_used_by|is_used_by|recommends|is...,SCR_004098|SCR_004834|SCR_001606|SCR_004098|SC...,NaN,NaN
AGSC,Ambystoma Genetic Stock Center,None,SCR_006372,nlx_152125,SCR_001062,PMID:16359543,"biomaterial supply resource,material resource,...",A. mexicanum stock center,AGSC,"RIN, Resource Information Network, embryo, lar...",...,NaN,https://orip.nih.gov/comparative-medicine/prog...,NaN,nlx_152125,"Sample type: embryo, larvae, adult",NaN,is_used_by|is_listed_by|is_listed_by|has_paren...,SCR_001421|SCR_004193|SCR_023186|SCR_001062,NaN,NaN
Addgene,Addgene,addgene,SCR_002037,nif-0000-11872,NaN,DOI:10.1093/nar/gku893,"organization portal,storage service resource,p...","Addgene Plasmid Database, Addgene Repository",NaN,"RIN, Resource Information Network, plasmid, mo...",...,Addgene,"https://ror.org/01nn1pw54, https://bio.tools/A...",https://www.addgene.org/terms-of-use/,"nif-0000-11872, grid.482682.2, ISNI: 0000 0004...",Sample type: PlasmidStart date: 2004 Type: Com...,Resource specific license,uses|is_used_by|is_used_by|is_used_by|is_used_...,SCR_013988|SCR_004834|SCR_001606|SCR_003890|SC...,NaN,NaN
BCBC,Beta Cell Biology Consortium,None,SCR_005136,nlx_144143,SCR_011756,NaN,"biomaterial supply resource,material resource,...",NaN,BCBC,"RIN, Resource Information Network, pancreatic ...",...,NaN,NaN,NaN,nlx_144143,"Sample type: Cell line, Embryonic stem cell li...",NaN,is_used_by|is_used_by|is_used_by|is_listed_by|...,SCR_004834|SCR_001421|SCR_001606|SCR_004193|SC...,NaN,NaN
BDSC,Bloomington Drosophila Stock Center,bdsc,SCR_006457,nif-0000-00241,SCR_011294,NaN,"biomaterial supply resource,material resource,...",Bloomington Drosophila Stock Center at Indiana...,BDSC,"RIN, Resource Information Network, disease mod...",...,NaN,https://orip.nih.gov/comparative-medicine/prog...,NaN,nif-0000-00241,"Sample type: Fly, Sequenced strain, Human dise...",NaN,is_used_by|is_listed_by|is_listed_by|is_relate...,SCR_001421|SCR_004193|SCR_023186|SCR_004193|SC...,NaN,NaN
CVCL,Cellosaurus,cellosaurus,SCR_013869,SCR_013869,SCR_012816,PMID:29805321,"database,data or information resource",NaN,NaN,"RIN, Resource Information Network, cell lines,...",...,Cellosaurus,NaN,https://www.expasy.org/terms-of-use,nif-0000-30108,NaN,CC BY-NC-ND,is_used_by|is_listed_by|has_parent_organizatio...,SCR_015894|SCR_023186|SCR_012816|SCR_001672|SC...,NaN,NaN
CWRU,Case Western Reserve University Cystic Fibrosi...,None,SCR_017238,SCR_017238,NaN,NaN,"biomaterial supply resource,material resource,...","CWR, Case Western Reserve University, Cystic F...",CWR,"RIN, Resource Information Network, cystic, fib...",...,NaN,NaN,NaN,NaN,Acknowledgement requested,NaN,is_listed_by,SCR_023186,NaN,NaN
DGGR,Kyoto Stock Center,None,SCR_008469,nif-0000-30415,NaN,NaN,"biomaterial supply resource,material resource,...","Drosophila Genomics and Genetic Resources, Dro...",DGGR,"RIN, Resource Information Network, organism su...",...,kyotofly,NaN,https://kyotofly.kit.jp/stocks/documents/Acqui...,nif-0000-30415,Please contact us when your institution reques...,Resource specific license,is_used_by|is_listed_by,SCR_001421|SCR_023186,NaN,NaN
EXRC,European Xenopus Resource Center,None,SCR_007164,nif-0000-38111,NaN,NaN,"biomaterial supply resource,material resource,...",NaN,EXRC,"RIN, Resource Information Ne

In [30]:
import bioregistry

for prefix in rrid_df.index:
    norm_prefix = bioregistry.normalize_prefix(prefix)
    if not norm_prefix:
        print("missing", prefix)
    else:
        print("found", prefix, norm_prefix)

missing IMSR
missing IMSR_TIGM
found Addgene addgene
missing RRRC
found MMRRC mmrrc
missing WB-STRAIN:WBStrain
missing IMSR_RBRC
missing IMSR_CRL
missing SAMN
missing ZIRC
missing BCBC
found SCR scr
missing IMSR_EM
missing IMSR_CMMR
missing IMSR_APB
missing AGSC
missing AB
found RGD rgd
found BDSC bdsc
found MGI mgi
found FlyBase flybase
missing NSRRC
missing EXRC
missing IMSR_HAR
missing XGSC
missing TSC
missing DGGR
missing IMSR_NIG
missing NXR
missing VDRC
found CVCL cellosaurus
missing SSCLBR
missing XEP
missing IMSR_CARD
missing IMSR_NM-KI
missing IMSR_JAX
missing IMSR_TAC
missing CWRU
missing IMSR_GPT
